In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
df = pd.read_csv("LLM-HistoricalData.csv")

os.makedirs("figures", exist_ok=True)


In [12]:
def convert_params(value):
    if isinstance(value, str):
        value = value.strip().upper()

        # Handle undisclosed / missing
        if value in ["NAN", "NONE", "", "UNDISCLOSED"]:
            return None

        # Extract numeric + unit
        if "B" in value:
            try:
                num = float(value.replace("B", "").strip())
                return num * 1e9
            except:
                return None

        if "M" in value:
            try:
                num = float(value.replace("M", "").strip())
                return num * 1e6
            except:
                return None

        if "T" in value:
            try:
                num = float(value.replace("T", "").strip())
                return num * 1e12
            except:
                return None

        # If already numeric in string
        try:
            return float(value)
        except:
            return None

    # If it's already numeric
    return value

# Apply conversion
df["Parameters_clean"] = df["Parameters"].apply(convert_params)


In [14]:
plt.figure(figsize=(10,6))
plt.plot(df["Year"], df["Parameters_clean"], marker="o")
plt.yscale("log")
plt.xlabel("Year")
plt.ylabel("Parameters (log scale)")
plt.title("LLM Parameter Growth Over Time")
plt.grid(True, alpha=0.3)
plt.savefig("figures/parameter_growth.png")
plt.close()


In [15]:
if "Compute" in df.columns:
    compute_df = df.dropna(subset=["Compute"])
    plt.figure(figsize=(12,6))
    plt.bar(compute_df["Model"], compute_df["Compute"])
    plt.xticks(rotation=90)
    plt.ylabel("Training FLOPs (log scale)")
    plt.yscale("log")
    plt.title("Compute Scaling Across Models")
    plt.tight_layout()
    plt.savefig("figures/compute_scaling.png")
    plt.close()

In [17]:
plt.figure(figsize=(7,7))
df["Architecture Family"].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title("Architecture Distribution")
plt.ylabel("")
plt.savefig("figures/architecture_distribution.png")
plt.close()

In [18]:
plt.figure(figsize=(12,6))
timeline = df.groupby(["Year", "Modality"]).size().reset_index(name="Count")

for modality in timeline["Modality"].unique():
    subset = timeline[timeline["Modality"] == modality]
    plt.plot(subset["Year"], subset["Count"], marker="o", label=modality)

plt.title("Modality Expansion Over Time")
plt.xlabel("Year")
plt.ylabel("Model Count")
plt.legend()
plt.grid(True, alpha=0.4)
plt.savefig("figures/modality_timeline.png")
plt.close()

print("All figures generated in /figures/")

All figures generated in /figures/
